In [2]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
import copy
import sys
from tqdm import tqdm
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

import pytrec_eval
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from haystack.pipelines import Pipeline
from haystack.nodes import BM25Retriever, ElasticsearchRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, SentenceTransformersRanker
from haystack.nodes.reader import FARMReader
from haystack.utils import print_answers
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../../')
import globals
from elastic_search_utils import elastic_utils
from haystack_utils.retrievers import BioASQ_Retriever
import bioasq_eval
import haystack_util

working_folder = globals.PATH.home + '/data/working_folder'
eval_home = globals.PATH.eval_home + '/'
gs_google_docs = eval_home + '/examples/aueb_google_docs/aueb_nlp-bioasq6b-submissions/'
index_name = globals.BIOASQ.index + 'working_folder'
model_id = 'doc_retrieval_test'

es = Elasticsearch(globals.ES.server)

# set document store
document_store = ElasticsearchDocumentStore()

Home path : /opt/bioasq/col-un-bioasq11
Eval path : /opt/bioasq/Evaluation-Measures


In [3]:
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/BioASQ-task11bPhaseA-testset3.json'
test_batch_json = json.load(open(test_batch_doc))

In [4]:
# set document store
document_store = ElasticsearchDocumentStore()
# create the retriever
retriever = BioASQ_Retriever(document_store = document_store)

# create the Sentence Transformer Ranker
ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

# create the Query Pipeline
pipeline = Pipeline()

# add bm25 retriever
pipeline.add_node(component=retriever, name="BM25Retriever", inputs=["Query"])
pipeline.add_node(component=ranker, name="Ranker", inputs=["BM25Retriever"])
#my_model = "deepset/roberta-base-squad2"
my_model = "deepset/roberta-base-squad2-covid"
#my_model = "twielema/biobert_v1.1_pubmed_squad_v2_entericdisease"
reader = FARMReader(model_name_or_path=my_model, use_gpu=True)
pipeline.add_node(component=reader, name="Reader", inputs=["Ranker"])

# run the pipeline
prediction = pipeline.run(query="Which gene is most frequently mutated in hereditary angioedema ?", params={"BM25Retriever": {"top_k": 100}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [5]:
prediction

{'query': 'Which gene is most frequently mutated in hereditary angioedema ?',
 'no_ans_gap': 17.294288128614426,
 'answers': [<Answer {'answer': 'type 2', 'type': 'extractive', 'score': 0.8722091317176819, 'context': 'used by either a C1 esterase inhibitor deficiency (type 1) or mutation (type 2). This leads to overproduction of bradykinin resulting in vasodilation,', 'offsets_in_document': [{'start': 152, 'end': 158}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['33780405'], 'meta': {'title': 'Recognition, Evaluation, and Management of Pediatric Hereditary Angioedema.', 'issue': '37(4)', 'pages': '218-223', 'abstract': "Hereditary angioedema (HAE) is a rare, often underrecognized genetic disorder caused by either a C1 esterase inhibitor deficiency (type 1) or mutation (type 2). This leads to overproduction of bradykinin resulting in vasodilation, vascular leakage, and transient nonpitting angioedema occurring most frequently in the face, neck, upper airway, abdo

In [6]:
def get_snipet_from_answer(pred):
    is_extractive = pred.type == 'extractive'
    answer = {}
    answer['document'] =  globals.BIOASQ.doc_relative_url + pred.document_ids[0]
    answer['text'] = pred.context
    answer['offsetInBeginSection'] = pred.meta['abstract'].rfind(pred.context)
    answer['offsetInEndSection'] = answer['offsetInBeginSection'] + len(pred.context)
    answer["beginSection"] = "abstract",
    answer["endSection"] = "abstract"
    return answer

In [21]:
for sample in tqdm(test_batch_json['questions'], position=0):
    prediction = pipeline.run(query=sample['body'], params={"BM25Retriever": {"top_k": 100}, "Ranker": {"top_k": 50}, "Reader": {"top_k": 50} })
    snippets = []
    doc_list_by_snnipets = []
    for ans in prediction['answers']:
        if ans.score > 0.4:
            snippet = get_snipet_from_answer(ans)
            doc_list_by_snnipets.append(snippet['document'])
            snippets.append(snippet)
            
    sample['snippets'] = snippets
    doc_list = [ globals.BIOASQ.doc_relative_url + d.id for d in prediction['documents'] ]
    #sample['documents'] = doc_list[0:10]
    sample['documents'] = list(set(doc_list_by_snnipets))
    
# mindlab base ans.score > 0.3:
# mindlab tns ans.score > 0.4 and only snnipets docs

  0%|                                                                                                                    | 0/90 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

  1%|█▏                                                                                                          | 1/90 [00:06<09:46,  6.59s/it]

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]

  2%|██▍                                                                                                         | 2/90 [00:14<11:10,  7.62s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 88/90 [10:18<00:13,  6.55s/it]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 89/90 [10:23<00:05,  5.89s/it]

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [10:30<00:00,  7.01s/it]


In [32]:
for q in test_batch_json['questions']:
    for s in q['snippets']:
        s['beginSection'] = s['beginSection'][0] 

In [33]:
model_id = 'bm25_ranker_reader_bioqa_ans_score_0_4_sentence_sim'
submission_file_name = f'{globals.PATH.home}/data/processed/{test_batch_doc.split("/")[-1].replace(".json","")}_model_{model_id}.json'

with open(submission_file_name, "w") as outfile:
    json.dump(test_batch_json, outfile, indent=2)

## Improve Passages

In [34]:
import editdistance
import nltk.data
import src.elastic_search_utils.elastic_utils as es_util

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [35]:
from rapidfuzz.distance import Levenshtein
Levenshtein.normalized_similarity("levenshtein", "levenshtein")

1.0

In [36]:
test_batch_json = json.load(open(submission_file_name))

In [37]:
for sample in tqdm(test_batch_json['questions'], position=0):
    for s in sample['snippets']:
        text_raw = s['text']
        doc_id = s['document'].replace(globals.BIOASQ.doc_relative_url,'')
        res = es_util.search_doc_by_id(doc_id)
        doc_title = res[doc_id]['title']
        doc_abstract =  res[doc_id]['abstract']
        sentences = tokenizer.tokenize(doc_abstract)
        # validate if the sentence is similar to the snippet
        for sentence in sentences:
            new_text = None
            if text_raw in sentence: 
                new_text = sentence
            else:
                if Levenshtein.normalized_similarity(sentence,text_raw) > 0.5: 
                    new_text = sentence
            if new_text:
                    s['text'] = new_text
                    s['offsetInBeginSection'] = doc_abstract.find(new_text)
                    s['offsetInEndSection'] = s['offsetInBeginSection'] + len(new_text)
                    s["beginSection"] = "abstract"
                    s["endSection"] = "abstract"
        # add title search

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:01<00:00, 54.40it/s]


In [38]:
import copy

In [39]:
copy_test_batch_json = copy.deepcopy(test_batch_json)

for sample in tqdm(test_batch_json['questions'], position=0):
    sample['documents'] = sample['documents'][0:10]
    sample['snippets'] = sample['snippets'][0:10]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 389162.23it/s]


In [40]:
with open(submission_file_name, "w") as outfile:
    json.dump(test_batch_json, outfile, indent=2)

In [41]:
submission_file_name

'/opt/bioasq/col-un-bioasq11/data/processed/BioASQ-task11bPhaseA-testset3_model_bm25_ranker_reader_bioqa_ans_score_0_4_sentence_sim.json'

# ChatGPT search

In [37]:
chat_gpt_fn = f'{globals.PATH.home}/data/11b_testset/ChatGPT.json'
chat_gpt = json.load(open(chat_gpt_fn, "r"))

In [1]:
chatgpt_questions = dict((elem['id'],elem) for elem in chat_gpt['questions'])

NameError: name 'chat_gpt' is not defined

In [70]:
for sample in tqdm(test_batch_json['questions'], position=0):
    doc_list = []
    snippet_list = []
    for s in sample['snippets']:
        text_raw = s['text']
        # validate if the sentence is similar to the snippet
        query = chatgpt_questions[sample['id']]['chatgpt-out']['short-ans']
        if Levenshtein.normalized_similarity(query,text_raw) > 0.1: 
            snippet_list.append(s)
            doc_list.append(s['document'])
    sample['snippets'] = snippet_list
    doc_list = set(doc_list)
    sample['documents'] = list(doc_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 8160.76it/s]


In [71]:
test_batch_json

{'questions': [{'id': '63f03006f36125a426000018',
   'type': 'summary',
   'body': 'What were the results of the DESTINY-Breast04 Trial?',
   'snippets': [{'document': 'http://www.ncbi.nlm.nih.gov/pubmed/35665782',
     'text': "CONCLUSIONS\nIn this trial involving patients with HER2-low metastatic breast cancer, trastuzumab deruxtecan resulted in significantly longer progression-free and overall survival than the physician's choice of chemotherapy.",
     'offsetInBeginSection': 2191,
     'offsetInEndSection': 2414,
     'beginSection': 'abstract',
     'endSection': 'abstract'},
    {'document': 'http://www.ncbi.nlm.nih.gov/pubmed/36386213',
     'text': 'DESTINY-Breast04 (NCT03734029) has demonstrated the antitumor activity of trastuzumab deruxtecan (T-DXd).',
     'offsetInBeginSection': 172,
     'offsetInEndSection': 277,
     'beginSection': 'abstract',
     'endSection': 'abstract'},
    {'document': 'http://www.ncbi.nlm.nih.gov/pubmed/31022475',
     'text': 'After evaluating

In [72]:
with open(submission_file_name, "w") as outfile:
    json.dump(test_batch_json, outfile, indent=2)

In [ ]:
copy_test_batch_json

In [48]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('deepset/roberta-base-squad2')

for sample in tqdm(test_batch_json['questions'], position=0):
    doc_list = []
    snippet_list = []
    query = chatgpt_questions[sample['id']]['chatgpt-out']['short-ans']
    en_query = model.encode(query)
    for s in sample['snippets']:
        text_raw = s['text']
        en_snippet = model.encode(text_raw)
        result = util.cos_sim(en_query, en_snippet)
        if result > 0.98:
            snippet_list.append(s)
            doc_list.append(s['document'])
    sample['documents'] = doc_list[0:10]
    sample['snippets'] = snippet_list[0:10]

Some weights of the model checkpoint at /home/andresr/.cache/torch/sentence_transformers/deepset_roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/andresr/.cache/torch/sentence_transformers/deepset_roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('deepset/roberta-base-squad2')

for sample in tqdm(test_batch_json['questions'], position=0):
    doc_list = []
    snippet_list = []
    query = chatgpt_questions[sample['id']]['chatgpt-out']['short-ans']
    en_query = model.encode(query)
    for s in sample['snippets']:
        text_raw = s['text']
        en_snippet = model.encode(text_raw)
        result = util.cos_sim(en_query, en_snippet)
        if result > 0.98:
            snippet_list.append(s)
            doc_list.append(s['document'])
    sample['documents'] = doc_list[0:10]
    sample['snippets'] = snippet_list[0:10]

In [49]:
test_batch_json

{'questions': [{'id': '63f03006f36125a426000018',
   'type': 'summary',
   'body': 'What were the results of the DESTINY-Breast04 Trial?',
   'snippets': [],
   'documents': []},
  {'id': '641ad941690f196b5100003d',
   'type': 'yesno',
   'body': 'Are LOQ and LOD the same?',
   'snippets': [{'document': 'http://www.ncbi.nlm.nih.gov/pubmed/21603916',
     'text': 'Real-time PCR (qPCR) is the principal technique for the quantification of pathogen biomass in host tissue, yet no generic methods exist for the determination of the limit of quantification (LOQ) and the limit of detection (LOD) in qPCR.',
     'offsetInBeginSection': 0,
     'offsetInEndSection': 236,
     'beginSection': 'abstract',
     'endSection': 'abstract'}],
   'documents': ['http://www.ncbi.nlm.nih.gov/pubmed/21603916']},
  {'id': '642a029d57b1c7a315000011',
   'type': 'summary',
   'body': 'Histone acetyltransferases (HATs) transfer acetyl groups from acetyl CoA to lysine residues on histones. What is the purpose of 

In [16]:
for sample in tqdm(test_batch_json['questions'], position=0):
    query = sample['body']
    if len(chatgpt_questions[sample['id']]['chatgpt-out']['queries']) > 1:
        query = ' '.join(chatgpt_questions[sample['id']]['chatgpt-out']['queries'])
    prediction = pipeline.run(query=query, params={"BM25Retriever": {"top_k": 50}, "Ranker": {"top_k": 30}, "Reader": {"top_k": 30} })
    print(query)
    snippets = []
    doc_list_by_snnipets = []
    for ans in prediction['answers']:
        if ans.score > 0.4:
            snippet = get_snipet_from_answer(ans)
            doc_list_by_snnipets.append(snippet['document'])
            snippets.append(snippet)

    sample['snippets'] = snippets
    doc_list = [ globals.BIOASQ.doc_relative_url + d.id for d in prediction['documents'] ]
    #sample['documents'] = doc_list[0:10]
    sample['documents'] = list(set(doc_list_by_snnipets))
    
# mindlab base ans.score > 0.3:
# mindlab tns ans.score > 0.4 and only snnipets docs

  0%|                                                                                                                    | 0/75 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

  1%|█▍                                                                                                          | 1/75 [00:06<07:39,  6.20s/it]

DESTINY-Breast04 trial results Trastuzumab deruxtecan efficacy in breast cancer Phase III study of trastuzumab deruxtecan in HER2-positive breast cancer Clinical trial of trastuzumab deruxtecan in breast cancer DESTINY-Breast04 trial design and outcomes HER2-targeted therapy in metastatic breast cancer Trastuzumab deruxtecan safety profile in breast cancer Antibody-drug conjugate therapy for breast cancer


Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 74/75 [05:38<00:04,  4.32s/it]

Tebentafusp clinical trials uveal melanoma Tebentafusp efficacy uveal melanoma Immunotherapy uveal melanoma Antigen-specific T-cell therapy uveal melanoma


Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [05:42<00:00,  4.57s/it]

Fenton reaction mechanism Hydroxyl radical generation in Fenton reaction Fenton reaction and oxidative stress Iron-catalyzed reactions in biological systems


In [18]:
test_batch_json

{'questions': [{'id': '63f03006f36125a426000018',
   'type': 'summary',
   'body': 'What were the results of the DESTINY-Breast04 Trial?',
   'snippets': [],
   'documents': []},
  {'id': '641ad941690f196b5100003d',
   'type': 'yesno',
   'body': 'Are LOQ and LOD the same?',
   'snippets': [],
   'documents': []},
  {'id': '642a029d57b1c7a315000011',
   'type': 'summary',
   'body': 'Histone acetyltransferases (HATs) transfer acetyl groups from acetyl CoA to lysine residues on histones. What is the purpose of this transfer?',
   'snippets': [],
   'documents': []},
  {'id': '6432fc0457b1c7a31500001f',
   'type': 'factoid',
   'body': 'Which gene is most frequently mutated in hereditary angioedema ?',
   'snippets': [],
   'documents': []},
  {'id': '64178edb690f196b51000025',
   'type': 'summary',
   'body': "What is casimersen's mechanism of action?",
   'snippets': [],
   'documents': []},
  {'id': '6402bf2b201352f04a000007',
   'type': 'summary',
   'body': 'What is the mechanism of

Downloading:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/691 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[[-0.7673091  -0.02201538  0.21489295 ...  0.4305086   0.9445836
   0.3237879 ]
 [ 0.04580967  0.2503939   1.1280602  ...  0.34196827  0.4122522
   0.14661177]]
